This section demonstrates the minimax-Q learning algorithm using a simple two-player zero-sum Markov game modeled after the game of soccer.
# Markov Games for Multi-Agent RL: Littman's Soccer Experiment  
**Based on "Markov Games as a Framework for Multi-Agent RL" (Littman, 1994)**  

---
---

## **Core Experiment Design**  

### 1. **Environment Definition (SOCCER)**  
- **Grid Structure**: 4x5 grid, with Player A starting at (2,1), Player B at (2,5), and ball possession randomly assigned.  
- **Objective**: Score by moving the ball into the opponent's goal (A left, B right). The environment resets to initial positions after a goal.  
- **Action Space**: Each player can choose actions `{N, S, E, W, Stand}`. Action execution order is random (50% chance Player A moves first).  
- **Ball Possession Rules**:  
  - Moving outside the grid results in the action failing, and the player remains in place.  
  - Moving to the opponent's position transfers possession to the stationary player, and the move fails.  
  - After scoring, ball possession is randomly assigned to either player.  

### 2. **Reward Mechanism**  
- **Zero-Sum Rewards**:  
  - The scorer receives $+1$, and the opponent receives $-1$.  
  - Discount factor $\gamma = 0.9$.  

### 3. **Algorithm Comparison**  
- **Minimax-Q**:  
  - Explicitly models the joint action space $(a_A, a_B)$.  
  - Policy update: $\max_{\pi_A} \min_{\pi_B} \mathbb{E}[Q(s,a_A,a_B)]$.  
- **Q-learning**:  
  - Assumes the opponent's policy is fixed (against random) or is an independent learner.  
  - Update rule: $Q(s,a_A) \leftarrow (1-\alpha)Q + \alpha[R + \gamma \max_{a'_A}Q(s',a'_A)]$.  

### 4. **Training Configuration**  
- **Learning Rate**: $\alpha$ decays from 0.2 to 0.01 (1 million training steps).  
- **Exploration Strategy**: $\epsilon$ decays from 1.0 according to $\epsilon_t = \epsilon_0 \exp(-\lambda t)$ to 0.9999954.  
- **Convergence Condition**: Variance of policy changes over 1000 consecutive steps is less than 0.001.  
- **Training Opponents**:  
  - MR/QR: Fixed random policy (uniform action selection).  
  - MM/QQ: Independent learners (separate Q-tables).  

## **Key Function Requirements**  
1. **State Encoder**:  
   - Input: Coordinates $(x_A, y_A, x_B, y_B)$ and ball possession flag.  
   - Output: Unique ID (e.g., $x_A + 4y_A + 16x_B + 64y_B + 256b$).  
2. **Asynchronous Action Executor**:  
   - Input: $(a_A, a_B)$.  
   - Output: Results of random-order execution (including collision detection).  
3. **Policy Mixer**:  
   - Input: $Q(s,\cdot,\cdot)$ matrix.  
   - Output: Nash equilibrium strategy pair $(\pi_A, \pi_B)$.  
4. **Convergence Detector**:  
   - Input: Historical policies $\{\pi_t\}_{t=1}^T$.  
   - Output: Sliding window variance (window size = 1000).  
5. **Adversarial Evaluator**:  
   - Input: Policy A vs. Policy B.  
   - Output: Win rate, average step length, and discounted reward difference.


#### References
- [Markov games as a framework for multi-agent reinforcement learning
Michael L. Littma](https://courses.cs.duke.edu/spring07/cps296.3/littman94markov.pdf)
- [Rock, Paper, Scissors Kaggle Competition](https://www.kaggle.com/c/rock-paper-scissors)

---

## **MARL Modeling Phases**  


### **Phase 1: Environment Dynamics Modeling**  
- **State Representation**:  
  - Coordinate encoding: $(x_A, y_A, x_B, y_B, ball\_owner)$ hashed into a unique ID  
  - Goal status: Dynamically detected based on position  
- **Conflict Handling**:  
  - Asynchronous action executor randomly orders $(a_A, a_B)$  
  - Positional conflict triggers ball possession transfer  

#### Code tips

In [ ]:
## **Phase 1: Environment Dynamics Modeling Pseudo Code**
class SoccerEnv:
    def _encode_state(self):
        # Coordinate hash encoding: (x_A, y_A, x_B, y_B, ball_owner) → int
        return hash(f"{self.pos_A}{self.pos_B}{self.ball_owner}")

    def _async_act(self, a_A, a_B):
        # Random execution order (50% chance to swap execution order)
        if np.random.rand() < 0.5:
            first_act, second_act = a_A, a_B
        else:
            first_act, second_act = a_B, a_A

        # Collision detection and ball possession transfer
        if self._check_collision(new_pos_first, pos_second):
            self.ball_owner = second_act.player_id  # Ball possession transfer


### **Phase 2: Value Function Design**  
- **Minimax-Q Update**:  
  $$Q(s,a_A,a_B) \leftarrow (1-\alpha)Q + \alpha[R + \gamma V^*(s')]$$  
  $$V^*(s') = \max_{\pi_A} \min_{a_B} \sum_{a_A} \pi_A(a_A)Q(s',a_A,a_B)$$  
- **Q-learning Update**:  
  $$Q(s,a_A) \leftarrow (1-\alpha)Q + \alpha[R + \gamma \max_{a'_A}Q(s',a'_A)]$$  


In [ ]:
## **Phase 2: Value Function Design Pseudo Code**
def minimax_q_update(Q, s, a_A, a_B, R, s_prime):
    V = solve_nash(Q[s_prime])  # Solve V*(s') using linear programming
    Q[s][a_A][a_B] = (1-alpha)*Q[s][a_A][a_B] + alpha*(R + gamma*V)

def q_learning_update(Q, s, a_A, R, s_prime):
    max_q_prime = np.max(Q[s_prime])  # Ignore opponent's actions
    Q[s][a_A] = (1-alpha)*Q[s][a_A] + alpha*(R + gamma*max_q_prime)

### **Phase 3: Policy Optimization**  
- **Minimax Strategy**: Solve for mixed strategy $\pi_A^* = \arg\max_{\pi} \min_{a_B} \mathbb{E}[Q]$ using linear programming  
- **Exploration Mechanism**: Maintain $\epsilon > 0.9$ during early training with $\epsilon$-greedy  



In [ ]:
## **Phase 3: Policy Optimization Pseudo Code**
def get_minimax_policy(Q, state):
    # Generate mixed strategy probability distribution (LP solution)
    payoff_matrix = Q[state]
    prob_A = solve_linear_program(payoff_matrix)  # Maximize minimum payoff
    return prob_A  # Example output: [0.3, 0.1, 0.2, 0.2, 0.2]

def epsilon_greedy(action_probs, epsilon):
    if np.random.rand() < epsilon:
        return np.random.choice(ACTION_SPACE)  # Explore
    else:
        return np.argmax(action_probs)         # Exploit

### **Phase 4: Training and Evaluation**  
1. **Training Protocol**:  
   - MM/QQ groups use parallel asynchronous updates (avoid locking)  
2. **Evaluation Methods**:  
   - Random opponent test: 10,000 steps to calculate win rate (including 0.1 probability of forced draw)  
   - Human strategy confrontation: Deterministic rules (intercept first + straight attack)  
   - Challenger test: Q-learning training against a frozen champion strategy  





In [ ]:
## **Phase 4: Training and Evaluation Pseudo Code**
def parallel_training(agent1, agent2):
    # Asynchronous parameter updates (lock to avoid conflicts)
    with threading.Lock():
        agent1.update_q()
        agent2.update_q()

def evaluate_vs_random(policy, n_episodes=1e5):
    win_count = 0
    for _ in range(n_episodes):
        if env.goal_scored == 'A':
            win_count += 1
        elif np.random.rand() < 0.1:  # Inject forced draw
            pass
    return win_count / n_episodes